
### Project description:

It is necessary to analyze the results of users acquisition to the mobile application for the period from 2019-05-01 to 2019-10-27. 

Input data:
- server log with information regarding app visits; 
- order information for the analysed period;
- advertising costs.

The object is to find the reasons of unefficient advertising compaign and give recomendations for the marketing department.

#### Data description:
Table visits_log_short (server log with information regarding application visits):

    1. User Id — unique user id
    2. Device — user's device
    3. Session start — date and time of the beginning user's session
    4. Session End — date and time of finishing the session
    5. Channel — advertisement source which attracted the user
    6. Region - user's country

Table orders_log_short (information about the orders):

    1. User Id — unique user id, who made an order
    2. Event Dt — дата и время покупки
    3. Revenue — revenue

Table costs_short (marketing costs):

    1. Channel — advertisement channel
    2. Dt — date
    3. Costs — costs for the advertising source in a specific day

## Assess data and data preparation

In [1]:
# import libraries

import pandas as pd
from datetime import datetime, timedelta
import seaborn as sns
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline

visits = pd.read_csv('/datasets/visits_info_short.csv')
orders = pd.read_csv('/datasets/orders_info_short.csv')
costs = pd.read_csv('/datasets/costs_info_short.csv')


Matplotlib is building the font cache; this may take a moment.


FileNotFoundError: [Errno 2] No such file or directory: '/datasets/visits_info_short.csv'

In [ ]:
#function for the quick datset analysis

def dataset (dataset):
    dataset.columns = [x.lower().replace(' ', '_') for x in dataset.columns.values]
    print(dataset.info())
    print('\n')
    print(dataset.describe())
    print('\n')
    print('Missed data')
    print(dataset.isnull().sum())
    print('\n')
    print(dataset.head(5))
    print('\n')
    print('Duplicates quantity', dataset.duplicated().sum())
    
dataset(visits)

In [ ]:
dataset(orders)

In [ ]:
dataset(costs)

In [ ]:
visits['session_start'] = pd.to_datetime(visits['session_start']) # bring the data to datetime format
visits['session_end'] = pd.to_datetime(visits['session_end'])

In [ ]:
orders['event_dt'] = pd.to_datetime(orders['event_dt']) # bring the data to datetime format

In [ ]:
costs['dt'] = pd.to_datetime(costs['dt']).dt.date # bring the data to datetime format

### Conclusion
We prepared data for the upcoming analysis by changing the format and checking the missing and duplicate values.

## The functions to calculate and analyze LTV, ROI, retention and conversion

Further we will define functions for creating user profiles, conversion, LTV, ROI and for visualizing these metrics — filter_data(), plot_retention(), plot_conversion() и plot_ltv_roi().

### The function for creating user profiles

In [ ]:

def get_profiles(sessions, orders, ad_costs):

    # sort sessions by user ID and engagement date
    # grouping by id and finding the parametres of first visits
    profiles = (
        sessions.sort_values(by=['user_id', 'session_start'])
        .groupby('user_id')
        .agg(
            {
                'session_start': 'first',
                'channel': 'first',
                'device': 'first',
                'region': 'first',
            }
        )
         # giving the name for the first session as 'first_ts'
        .rename(columns={'session_start': 'first_ts'})
        .reset_index()  # returning user_id from index
    )

    # defining the date of first visit for the cohort analysis
    # and the first day of month in which the first visit had place
    profiles['dt'] = profiles['first_ts'].dt.date
    profiles['month'] = profiles['first_ts'].astype('datetime64[M]')

    # adding an indicator of paying user
    profiles['payer'] = profiles['user_id'].isin(orders['user_id'].unique())


    # calculating the number if unique users
    # who have the same source and acquisition date
    new_users = (
        profiles.groupby(['dt', 'channel'])
        .agg({'user_id': 'nunique'})
         # giving the name for the number of users as 'unique_users'
        .rename(columns={'user_id': 'unique_users'})
        .reset_index()  # returning dt and channel from indexes
    )

    # uniting advertising costs and the number of acquired users
    # by date and acqusition channel
    ad_costs = ad_costs.merge(new_users, on=['dt', 'channel'], how='left')

    # dividing the advertisiment costs by the number of acquired users
    # saving the calculation results into column acquisition_cost (CAC)
    ad_costs['acquisition_cost'] = ad_costs['costs'] / ad_costs['unique_users']

    # adding acquisition cost to the profiles
    profiles = profiles.merge(
        ad_costs[['dt', 'channel', 'acquisition_cost']],
        on=['dt', 'channel'],
        how='left',
    )

    # as the users, attracted organically, are connected to the advertisiment costs
    # they have in the 'acquisition_cost' column 'NaN' values
    # so we can change these values for '0' as the acquisition cost is equal to zero
    profiles['acquisition_cost'] = profiles['acquisition_cost'].fillna(0)
    
    return profiles  # returning profiles with CAC

### The function to calculate users retention

In [ ]:

def get_retention(profiles, sessions, observation_date, horizon_days, dimensions = [], ignore_horizon = False):
    
    # решаем какие столбцы оставим для группировки удержания, столбец признака платящего пользователя остается всегда
    dimensions = ['payer'] + dimensions
    
    # оттсекаем всех пользователей, которые не могли "дожить" до нужного горизонта 
    # (присоединились позже чем observation_date - horizon)
    last_suitable_acquisition_date = observation_date
    if not ignore_horizon:
        last_suitable_acquisition_date = observation_date - timedelta(days = horizon_days - 1)
    result_raw = profiles.query('dt <= @last_suitable_acquisition_date')

    # forming the raw data massive
    result_raw = result_raw.merge(sessions[['user_id', 'session_start']], on = 'user_id', how = 'left')
    result_raw['lifetime'] = (result_raw['session_start'] - result_raw['first_ts']).dt.days
    
    # function to make grouping by desired dimension
    def group_by_dimensions(df, dims, horizon_days):     
        result = df.pivot_table(index = dims, columns = 'lifetime', values = 'user_id', aggfunc = 'nunique')     # building "triangle table" 
        cohort_sizes = df.groupby(dims).agg({'user_id': 'nunique'}).rename(columns = {'user_id': 'cohort_size'}) # defining the cohort sizes
        result = cohort_sizes.merge(result, on = dims, how = 'left').fillna(0)                                   # combining the cohort sizes and triangle table
        result = result.div(result['cohort_size'], axis = 0)                                                     # divide every column by the cohort size - to calculate retention rate %
        result = result[['cohort_size'] + list(range(horizon_days))]                                             # remain only the data before the analysis horizon and the column with the cohort size
        result['cohort_size'] = cohort_sizes                                                                     # redefining the cilumn with the cohorts sizes as in the previous step it became equal to '1'
        return result
    
    # calculating retention 
    result_grouped = group_by_dimensions(result_raw, dimensions, horizon_days)
    
    # calculating retention by all dimensions + date
    result_in_time = group_by_dimensions(result_raw, dimensions + ['dt'], horizon_days)
    
    # returning retention table and raw data (so we remain possibility to return to the beginning of analysis in case of any problems)
    return result_raw, result_grouped, result_in_time

### The function to calculate conversion

In [ ]:

def get_conversion(
    profiles,
    purchases,
    observation_date,
    horizon_days,
    dimensions=[],
    ignore_horizon=False,
):

    # excluding users outside of the analysis horizon
    last_suitable_acquisition_date = observation_date
    if not ignore_horizon:
        last_suitable_acquisition_date = observation_date - timedelta(
            days=horizon_days - 1
        )
    result_raw = profiles.query('dt <= @last_suitable_acquisition_date')

    # defining the date and first purchase time for every single user
    first_purchases = (
        purchases.sort_values(by=['user_id', 'event_dt'])
        .groupby('user_id')
        .agg({'event_dt': 'first'})
        .reset_index()
    )

    # adding purchasing data into profiles
    result_raw = result_raw.merge(
        first_purchases[['user_id', 'event_dt']], on='user_id', how='left'
    )

    # calculating lifetime value for every single purchase
    result_raw['lifetime'] = (
        result_raw['event_dt'] - result_raw['first_ts']
    ).dt.days

    # grouping by cohort, if there is nothing in dimensions
    if len(dimensions) == 0:
        result_raw['cohort'] = 'All users'
        dimensions = dimensions + ['cohort']

    # function for grouping table by desired dimensions
    def group_by_dimensions(df, dims, horizon_days):
        # building "triangle" conversion table
        result = df.pivot_table(
            index=dims, columns='lifetime', values='user_id', aggfunc='nunique'
        )
        # calculating cumulative amount for every line
        result = result.fillna(0).cumsum(axis = 1)
        # calculating cohort size
        cohort_sizes = (
            df.groupby(dims)
            .agg({'user_id': 'nunique'})
            .rename(columns={'user_id': 'cohort_size'})
        )
        # adding the cohort size to the conversion table
        result = cohort_sizes.merge(result, on=dims, how='left').fillna(0)
        # dividing every 'cell' in a line by the cohort size
        # and getting the conversion rate
        result = result.div(result['cohort_size'], axis=0)
        # excluding all lifetimes after the analysis horizon
        result = result[['cohort_size'] + list(range(horizon_days))]
        # recovering the cohort size
        result['cohort_size'] = cohort_sizes
        return result

    # getting the conversion table
    result_grouped = group_by_dimensions(result_raw, dimensions, horizon_days)

    # for the dynamic of conversion table drop 'cohort' from dimensions
    if 'cohort' in dimensions: 
        dimensions = []

    # getting the conversion dynamics table
    result_in_time = group_by_dimensions(
        result_raw, dimensions + ['dt'], horizon_days
    )

    # returning both tables and raw data
    return result_raw, result_grouped, result_in_time

### The function to calculte LTV and ROI

In [ ]:

def get_ltv(
    profiles,
    purchases,
    observation_date,
    horizon_days,
    dimensions=[],
    ignore_horizon=False,
):

    # excluding users outside of the analysis horizon
    last_suitable_acquisition_date = observation_date
    if not ignore_horizon:
        last_suitable_acquisition_date = observation_date - timedelta(
            days=horizon_days - 1
        )
    result_raw = profiles.query('dt <= @last_suitable_acquisition_date')
    # add data about ourchases to profiles
    result_raw = result_raw.merge(
        purchases[['user_id', 'event_dt', 'revenue']], on='user_id', how='left'
    )
    # calculating users lifetime for every purchase
    result_raw['lifetime'] = (
        result_raw['event_dt'] - result_raw['first_ts']
    ).dt.days
    # group by cohort, if there is nothing in dimensions
    if len(dimensions) == 0:
        result_raw['cohort'] = 'All users'
        dimensions = dimensions + ['cohort']

    # function for grouping table by desired dimensions
    def group_by_dimensions(df, dims, horizon_days):
        # building "triangle" revenue table
        result = df.pivot_table(
            index=dims, columns='lifetime', values='revenue', aggfunc='sum'
        )
        # calculating cumulative revenue amount
        result = result.fillna(0).cumsum(axis=1)
        # calculating cohort size
        cohort_sizes = (
            df.groupby(dims)
            .agg({'user_id': 'nunique'})
            .rename(columns={'user_id': 'cohort_size'})
        )
        # combine the cohort sizes and the revenue table
        result = cohort_sizes.merge(result, on=dims, how='left').fillna(0)
        # calculating LTV: divide each "cell" in a row by the size of the cohort
        result = result.div(result['cohort_size'], axis=0)
        # exclude all liftimes exceeding the analysis horizon
        result = result[['cohort_size'] + list(range(horizon_days))]
        # redefining cohort size
        result['cohort_size'] = cohort_sizes

        # saving user data and CAC values to the dataframe,
        # adding parameters from dimensions
        cac = df[['user_id', 'acquisition_cost'] + dims].drop_duplicates()

        # calculating average CAC by parametres from dimensions
        cac = (
            cac.groupby(dims)
            .agg({'acquisition_cost': 'mean'})
            .rename(columns={'acquisition_cost': 'cac'})
        )

        # calculate ROI: ratio of LTV to CAC
        roi = result.div(cac['cac'], axis=0)

        # dropping lines with infinite ROI
        roi = roi[~roi['cohort_size'].isin([np.inf])]

        # redefining cohort size in ROI table
        roi['cohort_size'] = cohort_sizes

        # adding CAC to the ROI table
        roi['cac'] = cac['cac']

        # in the final table we leave the cohort sizes, CAC
        # and ROI in lifetimes not exceeding the analysis horizon
        roi = roi[['cohort_size', 'cac'] + list(range(horizon_days))]

        # returning tables for LTV and ROI
        return result, roi

    # getting the tables for LTV and ROI
    result_grouped, roi_grouped = group_by_dimensions(
        result_raw, dimensions, horizon_days
    )

    # for tables with historical data excluding 'cohort' from dimensions
    if 'cohort' in dimensions:
        dimensions = []

    # getting the LTV and ROI dynamic tables
    result_in_time, roi_in_time = group_by_dimensions(
        result_raw, dimensions + ['dt'], horizon_days
    )

    return (
        result_raw,  # raw data
        result_grouped,  # LTV table
        result_in_time,  # LTV dynamic table
        roi_grouped,  # ROI table
        roi_in_time,  # ROI dynamic table
    )

### Function for smoothing the frame

In [ ]:

def filter_data(df, window):
    # application of an mooving average for every column
    for column in df.columns.values:
        df[column] = df[column].rolling(window).mean() 
    return df 

### Function for visualizing retention

In [ ]:
# function for visualizing retention

def plot_retention(retention, retention_history, horizon, window=14):

    # setting the grid size for graphs
    plt.figure(figsize=(15, 10))

    # excluding cohort sizes and retention of the first day
    retention = retention.drop(columns=['cohort_size', 0])
    # leaving only the necessary lifetime in the dynamics table
    retention_history = retention_history.drop(columns=['cohort_size'])[
        [horizon - 1]
    ]

    # if only payer is in the retention table indexes,
    # adding the second attribute — cohort
    if retention.index.nlevels == 1:
        retention['cohort'] = 'All users'
        retention = retention.reset_index().set_index(['cohort', 'payer'])

    # in the graph table — two columns and two rows, four cells
    # in the first one, we build retention curves for paying users
    ax1 = plt.subplot(2, 2, 1)
    retention.query('payer == True').droplevel('payer').T.plot(
        grid=True, ax=ax1
    )
    plt.legend()
    plt.xlabel('Lifetime')
    plt.title('Paying user retention')

    # construct retention curves for non-paying in the second cell
    # vertical axis — from the graph from the first cell
    ax2 = plt.subplot(2, 2, 2, sharey=ax1)
    retention.query('payer == False').droplevel('payer').T.plot(
        grid=True, ax=ax2
    )
    plt.legend()
    plt.xlabel('Lifetime')
    plt.title('Non-paying user retention')

    # the dynamics of paying retention is in the third cell
    ax3 = plt.subplot(2, 2, 3)
    # getting the column names for the pivot table
    columns = [
        name
        for name in retention_history.index.names
        if name not in ['dt', 'payer']
    ]
    # filter data and build visualization
    filtered_data = retention_history.query('payer == True').pivot_table(
        index='dt', columns=columns, values=horizon - 1, aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax3)
    plt.xlabel('Acquisition date')
    plt.title(
        'Paying user retention dynamics на {} day'.format(
            horizon
        )
    )

    # the dynamics of non-paying retention is in the fourth cell
    ax4 = plt.subplot(2, 2, 4, sharey=ax3)
    # filter data and build visualization
    filtered_data = retention_history.query('payer == False').pivot_table(
        index='dt', columns=columns, values=horizon - 1, aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax4)
    plt.xlabel('Acquisition date')
    plt.title(
        'User retention dynamics на {} day'.format(
            horizon
        )
    )
    
    plt.tight_layout()
    plt.show() 


### Conversion visualization function

In [ ]:
# conversion visualization function

def plot_conversion(conversion, conversion_history, horizon, window=14):

    # setting the grid size for charts
    plt.figure(figsize=(15, 5))

    # excluding cohort size
    conversion = conversion.drop(columns=['cohort_size'])
    # we leave only the necessary lifetime in the dynamics table
    conversion_history = conversion_history.drop(columns=['cohort_size'])[
        [horizon - 1]
    ]

    # the first chart - conversion lines
    ax1 = plt.subplot(1, 2, 1)
    conversion.T.plot(grid=True, ax=ax1)
    plt.legend()
    plt.xlabel('Lifetime')
    plt.title('User conversion')

    # the first chart - conversion dynamics
    ax2 = plt.subplot(1, 2, 2, sharey=ax1)
    columns = [
        # the columns of the pivot table will be all the columns of the index, except for the date
        name for name in conversion_history.index.names if name not in ['dt']
    ]
    filtered_data = conversion_history.pivot_table(
        index='dt', columns=columns, values=horizon - 1, aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax2)
    plt.xlabel('Acquisition date')
    plt.title('User conversion dynamics на {} day'.format(horizon))

    plt.tight_layout()
    plt.show()

### The function for LTV and ROI visualization

In [ ]:
# function for LTV and ROI visualization

def plot_ltv_roi(ltv, ltv_history, roi, roi_history, horizon, window=14):

    plt.figure(figsize=(20, 10))

    # excluding from ltv table cohort size
    ltv = ltv.drop(columns=['cohort_size'])
    # saving in ltv dynamic table only relevant lifetime
    ltv_history = ltv_history.drop(columns=['cohort_size'])[[horizon - 1]]

    # save acqusition cost to the separate dataframe
    cac_history = roi_history[['cac']]

    # excluding from roi table cohort sizes and cac
    roi = roi.drop(columns=['cohort_size', 'cac'])
    # saving in roi dynamic table only relevant lifetime
    roi_history = roi_history.drop(columns=['cohort_size', 'cac'])[
        [horizon - 1]
    ]

    # LTV curves
    ax1 = plt.subplot(2, 3, 1)
    ltv.T.plot(grid=True, ax=ax1)
    plt.legend()
    plt.xlabel('Lifetime')
    plt.title('LTV')

    # LTV dynamics
    ax2 = plt.subplot(2, 3, 2, sharey=ax1)
    # the columns of the pivot table will be all the columns of the index, except for the date
    columns = [name for name in ltv_history.index.names if name not in ['dt']]
    filtered_data = ltv_history.pivot_table(
        index='dt', columns=columns, values=horizon - 1, aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax2)
    plt.xlabel('Acquisition date')
    plt.title('Динамика LTV пользователей на {}-й день'.format(horizon))

    # CAC dynamics
    ax3 = plt.subplot(2, 3, 3, sharey=ax1)
    # the columns of the pivot table will be all the columns of the index, except for the date
    columns = [name for name in cac_history.index.names if name not in ['dt']]
    filtered_data = cac_history.pivot_table(
        index='dt', columns=columns, values='cac', aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax3)
    plt.xlabel('Acquisition date')
    plt.title('Acquistion cost dynamics')

    # ROI curves
    ax4 = plt.subplot(2, 3, 4)
    roi.T.plot(grid=True, ax=ax4)
    plt.axhline(y=1, color='red', linestyle='--', label='Breakeven level')
    plt.legend()
    plt.xlabel('Lifetime')
    plt.title('ROI')

    # ROI dynamics
    ax5 = plt.subplot(2, 3, 5, sharey=ax4)
    # the columns of the pivot table will be all the columns of the index, except for the date
    columns = [name for name in roi_history.index.names if name not in ['dt']]
    filtered_data = roi_history.pivot_table(
        index='dt', columns=columns, values=horizon - 1, aggfunc='mean'
    )
    filter_data(filtered_data, window).plot(grid=True, ax=ax5)
    plt.axhline(y=1, color='red', linestyle='--', label='Уровень окупаемости')
    plt.xlabel('Acquisition date')
    plt.title('Users ROI dynamic for the {} day'.format(horizon))

    plt.tight_layout()
    plt.show()

## EDA

Let's define the last and the first date of users acquisition.

In [ ]:
print(visits['session_start'].min())

In [ ]:
print(visits['session_start'].max())

The analysis is carried out as of November 1, 2019, the data period corresponds to expectations - in the terms of the project it was said that the analysis is carried out for users attracted from May to October 27, 2019.

Let's build user profiles, immediately excluding users who came organically and are not subject of marketing costs.

In [ ]:
profiles = get_profiles(visits, orders, costs)

# excluding organic traffic users
profiles = profiles.query('channel != "organic"')
profiles['dt'] = pd.to_datetime(profiles['dt'])

profiles

Let's define from which countries users are coming and what is the conversion to paying user in every specific country.

In [ ]:
#function for calculating the shares and the number of paying customers by dimension

def payers_count (dimension):
    print(
    profiles.groupby(dimension)
    .agg({'payer':'sum'})
    .sort_values(by='payer', ascending=False)
    )
    with pd.option_context('display.float_format', '{:.2%}'.format):
        display(
        profiles.groupby(dimension)
        .agg({'payer':'mean'})
        .sort_values(by='payer', ascending=False)
        )

dimension = ['region']
payers_count(dimension=dimension)

The highest conversion rate is for users which came from the US.

In [ ]:
dimension = ['device']
payers_count(dimension=dimension)

The highest conversion rate is for users with IPhone device, the lowest conversion is observed on PC devices.

In [ ]:
dimension = ['channel']
payers_count(dimension=dimension)

### Conclusion

1. The countries from which visitors come: USA, Great Britain, France and Germany. At the same time, most of the paying users come from the USA.

2. Users use the following devices: Mac, iPhone, Android, PC. Mac and iPhone users have the largest conversion.

3. Top 3 channels for acqusition of paying users: FaceBoom, AdNonSense, lambdaMediaAds - the conversion rate on each of these channels is more than 10%. At the same time, most new visitors come through the channels FaceBoom, TipTop, organic.

## Marketing

In [ ]:
print('{:.0f}'.format(costs['costs'].sum()))

The total spending on the advertising is $ 105,497. Let's calculate average CAC:

In [ ]:
print('{:.2f}'.format(profiles['acquisition_cost'].mean()))

In [ ]:
print('{:.2f}'.format(profiles['acquisition_cost'].median()))

Average CAC for the observing period is $ 1.13, while the median one is $ 1.08.

In [ ]:
with pd.option_context('display.float_format', '{:.0f}'.format):
    display(costs.groupby('channel').agg(total_costs=('costs','sum')).sort_values(by='total_costs', ascending=False))

Most of the money was spent on TipTop and FaceBoom. Taking into account the average cost of attraction, these are also the most expensive sources - the average cost of attracting customers through these channels is more than the median one:

In [ ]:
with pd.option_context('display.float_format', '{:.2f}'.format):
    display(profiles.groupby('channel').agg(average_CAC =('acquisition_cost','mean')).sort_values(by='average_CAC', ascending=False))

In [ ]:
# creating CAC visualization for the five most expensive channels
top3channels = ['TipTop','FaceBoom', 'AdNonSense', 'lambdaMediaAds', 'WahooNetBanner']
profiles_top3channels = profiles.query('channel in @top3channels')

profiles_top3channels.pivot_table(
    index='dt', columns='channel', values='acquisition_cost', aggfunc='mean'
).plot(grid=True, figsize=(10, 5))
plt.ylabel('CAC, $')
plt.xlabel('Acquisition date')
plt.title('CAC dynamics by channels')
plt.show()

Рекламный бюджет по пользователям в TipTop увеличивался каждый месяц вплоть до сентября - видимо, у компании была особая стратегия для этой платформы.

In [ ]:
costs['costs_month'] = costs['dt'].astype('datetime64[M]').dt.date
costs_by_month = costs.groupby('costs_month')['costs'].sum().reset_index()
costs_by_month

In [ ]:
plt.figure(figsize = (10, 5))
ax = sns.barplot(x="costs_month", y="costs", color='blue', data=costs_by_month)
plt.title('Расходы помесячно')
ax.set_xlabel('Months')
ax.set_ylabel('Costs')
plt.show()

The maximum advertisement spent occured in September, October and August.

### Conclusion

The acquisition cost for the TipTop channel has been increasing gradually for the five months. The costs for this channel take more than a half of all advertising budget.

## Analysis of the total return on advertisement costs for the users acquisition

### Analysis of the total return on advertisement costs

The defining of the moment and horizon of data analysis is needed. According to the training case, the analysis is performing on the November 1st, 2019. We will set a two-week analysis horizon, because it is considered that payback should come no later than two weeks after the user acquisition.

In [ ]:
observation_date = datetime(2019, 10, 31).date()  # moment of analysis
horizon_days = 14  # analysis horizon

For the beginning we will estimate the general return on advertisement costs. By using the functions get_ltv() and plot_ltv_roi(), we will calculate and visualize LTV and ROI.

In [ ]:
# calculating LTV and ROI
ltv_raw, ltv_grouped, ltv_history, roi_grouped, roi_history = get_ltv(
    profiles, orders, observation_date, horizon_days
)

# building visualization
plot_ltv_roi(ltv_grouped, ltv_history, roi_grouped, roi_history, horizon_days)

1. Advertising does not pay off. The ROI at the end of two weeks is slightly above 80%.
2. CAC increases all the time incrementally from month to month. Most likely, this is due to a consistent increase in the advertising budget for TipTop - out of the top 5 largest channels in terms of total spending, only spending on it has a similar dynamics with the general schedule.
3. LTV is affected by the seasonal factor, but this indicator is quite stable. So, it's not about the negative dynamic of quality of users.

To understand the reasons, we will go through all the available characteristics of users — the country, the source and the device of the first visit.

### Analysis of the return on advertisement costs by countries

In [ ]:
dimensions = ['region']

ltv_raw, ltv_grouped, ltv_history, roi_grouped, roi_history = get_ltv(
    profiles, orders, observation_date, horizon_days, dimensions=dimensions
)

plot_ltv_roi(
    ltv_grouped, ltv_history, roi_grouped, roi_history, horizon_days, window=14
) 

The breakdown by countries shows a clear problem with users from the US:

1. Of all countries, advertising does not pay off only in the US - at the end of two weeks, the ROI for this country is about 70%, while other countries are close to 150%.
2. The cost of attraction is stable for all countries except the USA, where this indicator is constantly growing and reaches a constant level only from mid-September 2019.
3. Users from the USA have the highest LTV. Then go Great Britain and Germany. Users from France bring the least revenue to the application.

In [ ]:
conversion_raw, conversion, conversion_history = get_conversion(
    profiles, orders, observation_date, horizon_days, dimensions=['region']
)

# creating heatmap by conversion table
plt.figure(figsize=(15, 5))
sns.heatmap(conversion.drop(columns=['cohort_size']), linewidths=.3, annot=True, fmt='.2%')
plt.title('Conversion heatmap by contries')
plt.show() 

In [ ]:
# creating conversion curve

report = conversion.drop(columns=['cohort_size'])
report.T.plot(grid=True, xticks=list(report.columns.values), figsize=(10, 5))
plt.title('Conversion curve')
plt.show()

Throughout the entire horizon of the analysis, the conversion of users from the United States is about 2 times higher than the conversion of those from other countries, although this is the only region where advertising does not pay off.

In [ ]:
# buiding retention by countries

retention_raw, retention_grouped, retention_history = get_retention(
    profiles, visits, observation_date, horizon_days, dimensions=dimensions
)

plot_retention(retention_grouped, retention_history, horizon_days)

Users from the USA are consistently poorly retained. Retention of users from the USA is about 10% lower than for other countries. 

### Analysis of the return on advertisement costs by countries by devices

In [ ]:
# return by device dimension

dimensions = ['device']

ltv_raw, ltv_grouped, ltv_history, roi_grouped, roi_history = get_ltv(
    profiles, orders, observation_date, horizon_days, dimensions=dimensions
)

plot_ltv_roi(
    ltv_grouped, ltv_history, roi_grouped, roi_history, horizon_days, window=14
) 

PC users are steadily paying off, Mac and Iphone - definitely not since July 2019. In May 2019, advertising budgets for users of all devices were approximately at the same level, but since June, the cost of attracting Mac and Iphone users has been growing equally. Users of Android mobile devices are also trending with them, and the budget for PC users has increased slightly. 

In [ ]:
# retention by device dimension

retention_raw, retention_grouped, retention_history = get_retention(
    profiles, visits, observation_date, horizon_days, dimensions=dimensions
)

plot_retention(retention_grouped, retention_history, horizon_days)

The retention dynamics for users of all devices is approximately the same, for a PC it is even higher than other devices. It's probably all about conversion. Let's take a closer look at the retention of users through channels using a heat map.

In [ ]:
conversion_raw, conversion, conversion_history = get_conversion(
    profiles, orders, observation_date, horizon_days, dimensions=['device']
)

# creating heatmap by a conversion table
plt.figure(figsize=(15, 5))
sns.heatmap(conversion.drop(columns=['cohort_size']), linewidths=.3, annot=True, fmt='.2%')
plt.title('Heatmap in breakdown by devices')
plt.show() 

The highest conversion rate is among Mac device users, the lowest is among PC users. Perhaps the current advertising companies are not optimized for this type of device.

In [ ]:
# conversion rate by devices

conversion_raw, conversion_grouped, conversion_history = get_conversion(
    profiles, orders, observation_date, horizon_days, dimensions=dimensions
)

plot_conversion(conversion_grouped, conversion_history, horizon_days)

all users are converted approximately the same in dynamics, but the conversion rate for PC users is lower. The decline in the conversion rate of PC users attracted in the second half of July 2019 is particularly noticeable.

### Analysi of the return on advertising expences by channels

Let's check the sources of acquisition (channels). It is possible there is an unsuccessful advertising campaign in one channel.

In [ ]:
# calculating LTV and ROI

dimensions=['channel']

ltv_raw, ltv, ltv_history, roi, roi_history = get_ltv(
    profiles, orders, observation_date, horizon_days, dimensions
)

display(roi)  # ROI table

# LTV table
display(ltv)

# LTV curves
report = ltv.drop(columns=['cohort_size'])
report.T.plot(grid=True, figsize=(10, 5), xticks=list(report.columns.values))
plt.title('LTV by sources')
plt.ylabel('LTV, $')
plt.xlabel('Lifetime')
plt.show()

Users came from TipTop and lambdaMediaAds bring the major part of revenue.

In [ ]:
roi_history

In [ ]:
# building the visualization of CAC changing history

roi_history.pivot_table(
    index='dt', columns='channel', values='cac', aggfunc='mean'
).plot(grid=True, figsize=(10, 5))

plt.ylabel('CAC, $')
plt.xlabel('Acquisition date')
plt.title('САС dynamics by channels')
plt.show()

In [ ]:
# conversion by advert. channels

conversion_raw, conversion_grouped, conversion_history = get_conversion(
    profiles, orders, observation_date, horizon_days, dimensions=dimensions
)

plot_conversion(conversion_grouped, conversion_history, horizon_days)

It seems that some experiments have been carried out with advertising companies on the 'lambda Media Ads' channel - the conversion of users on the 14th day is constantly changing throughout the year.

In [ ]:
conversion_raw, conversion, conversion_history = get_conversion(
    profiles, orders, observation_date, horizon_days, dimensions
)

# creating heatmap according to conversion table
plt.figure(figsize=(15, 5))
sns.heatmap(conversion.drop(columns=['cohort_size']), linewidths=.3, annot=True, fmt='.2%')
plt.title('Conversion heatmep by channels')
plt.show() 

We can single out the most inefficient channels in terms of user conversion: OppleCreativeMedia, MediaTornade, LeapBob, YRabbit. The conversion rate for the TipTop channel, which is the most expensive in terms of the cost of attracting a user, is lower than for the Ad Non Sense and FaceBoom, lambdaMediaAd channels, where the user's advertising budget is approximately the same.

In [ ]:
# visualizing of retention by advertising channels

retention_raw, retention_grouped, retention_history = get_retention(
    profiles, visits, observation_date, horizon_days, dimensions
)

plot_retention(retention_grouped, retention_history, 14) 

Users from AdNonSense and FaceBoom show worse retention than others. 

In [ ]:
# building heatmap for retention by channels

plt.figure(figsize=(15, 6))
sns.heatmap(retention_grouped.drop(columns=['cohort_size', 0]), linewidths=.015, annot=True, fmt='.2%')
plt.title('Retention heatmap by channels')
plt.show()

For all paying users, the retention on the first day is about 50% and above, except for users who came through AdNonSense and FaceBoom. For these channels the retention on the first day does not reach even 30%.

### Conclusion

The most paying users come from the USA, however, this is the only region where advertising does not pay off due to poor user retention. Perhaps the quality of advertising companies used in these markets suffers. Let's see which advertising channels users from the USA come from:

In [ ]:
profiles_us = profiles.query('region == "United States"')

with pd.option_context('display.float_format', '{:.2}'.format):
    display(
    profiles_us.groupby('channel')
    .agg({'payer':'count'})
    .sort_values(by='payer', ascending=False)
)

It all fits together: the bulk of users come through the Facebook channel, which has the worst retention rate and through the TipTop channel, which is the most expensive.

## Conclusion

### The reasons of inefficient acquisition campaign

- The low retention for the users from the US, which are the most siginificant part of the paying customers

- The major channels for acquiring US users are FaceBoom and TipTop. Faceboom gives the users with the lowest retention meanwhile TipTop gives the most expensive users.

### The recomendations to the marketing department to increase effiecincy

Based on the results of the analysis, the following recommendations can be given for the marketing department:

- review the advertising strategy for AdNonSense and FaceBoom channels, possibly abandon them as channels that bring the most "leaked" users

- work on retaining users from the USA. It is possible to attract users of these countries through advertising channels with a lower conversion rate, but a better audience for our application, for example, TipTop. Users from the USA paid off for the last time in May 2019 - it's worth seeing what the advertising companies were in this month

- ask testers to check the effectiveness of the application on Apple and Android devices

- inform the product manager about the general situation with user retention - you can attract users indefinitely, but there are much more growth points in user retention. Perhaps new features should be added to the application for greater user engagement

- introduce interim control of the results of the marketing department, pay more attention to metrics.